In [25]:
import pandas as pd
from pathlib import Path

In [26]:
base_dir = Path("../../analysis/")
clean_dir = base_dir / "fleurs"
noisy_dir = base_dir / "noisy_fleurs_babble"

In [27]:
langs = [ 'en_es', 'en_fr', 'en_pt', 'en_it', 'en_de', 'en_nl',  'en_zh',     
                    'es_en', 'fr_en', 'pt_en', 'it_en', 'de_en',  'zh_en', ]

In [28]:
def compute_diffs(clean_file, noisy_file, fill_missing_with_zero=False):
    """Compute differences between clean and noisy results: clean - noisy"""
    
    clean_df = pd.read_csv(clean_file)
    noisy_df = pd.read_csv(noisy_file)

    # Merge with clear suffixes
    merged = clean_df.merge(
        noisy_df, on='system', how='outer', suffixes=('_clean', '_noisy'), indicator=True
    )

    # Matching column names
    clean_cols = list(merged.filter(like='_clean').columns)
    noisy_cols = [c.replace('_clean', '_noisy') for c in clean_cols]

    # Keep only columns that exist in both
    common_pairs = [(c_col, n_col) for c_col, n_col in zip(clean_cols, noisy_cols) if n_col in merged.columns]

    # Start output table
    out = merged[['system', '_merge']].copy()

    # Compute diff: clean - noisy
    for c_col, n_col in common_pairs:
        base = c_col[:-6]  # remove trailing '_clean'
        diff_col = f'{base}_diff'

        # Convert to numeric
        clean_vals = pd.to_numeric(merged[c_col], errors='coerce')
        noisy_vals = pd.to_numeric(merged[n_col], errors='coerce')

        if fill_missing_with_zero:
            clean_vals = clean_vals.fillna(0)
            noisy_vals = noisy_vals.fillna(0)
        clean_vals = clean_vals * 100
        out[diff_col] = (100*(clean_vals - noisy_vals)/clean_vals).round(2)

    return out

In [29]:
result = {}
for lang in langs:
    clean_file = clean_dir / f"fleurs_{lang}.csv"
    noisy_file = noisy_dir / f"noisy_fleurs_babble_{lang}.csv"
    
    diff_df  = compute_diffs(clean_file, noisy_file)
    result[lang] = diff_df
    noisy_df = pd.read_csv(noisy_file)
    system_order = noisy_df['system'].tolist()
    diff_df = diff_df.set_index('system').reindex(system_order).reset_index()
    out_file = noisy_dir / f"diff_{lang}.csv"
    diff_df.to_csv(out_file, index=False)

In [30]:
for k, v in result.items():
    print(k)
    print(v, '\n\n')

en_es
               system      _merge  LinguaPy_diff  \
0       aya_canary-v2        both         104.67   
1     aya_owsm4.0-ctc        both         113.00   
2     aya_seamlessm4t        both         101.75   
3         aya_whisper        both         121.50   
4           canary-v2        both         114.50   
5           desta2-8b        both         136.00   
6     gemma_canary-v2        both         114.50   
7   gemma_owsm4.0-ctc        both         136.50   
8   gemma_seamlessm4t        both         103.50   
9       gemma_whisper        both         135.00   
10        owsm4.0-ctc        both         106.80   
11     phi4multimodal        both         103.45   
12      qwen2audio-7b        both         101.50   
13        seamlessm4t        both         113.75   
14            spirelm        both         102.25   
15    tower_canary-v2        both         118.50   
16  tower_owsm4.0-ctc        both         100.00   
17  tower_seamlessm4t        both         103.00   
18    

In [24]:
clean_df = pd.read_csv(clean_file)
print(clean_df)

   direction             system  LinguaPy  QEMetricX_24-Strict-linguapy  \
0      zh_en          canary-v2   65.6085                       24.3884   
1      zh_en    tower_canary-v2   43.9153                       19.0280   
2      zh_en      aya_canary-v2    9.8413                       18.8023   
3      zh_en        owsm4.0-ctc    3.4921                       19.5131   
4      zh_en          desta2-8b    0.6349                        4.3109   
5      zh_en            whisper    0.2116                        5.9495   
6      zh_en      qwen2audio-7b    5.0794                        3.9521   
7      zh_en     phi4multimodal    5.7143                        3.7811   
8      zh_en  voxtral-small-24b    0.2116                        3.0309   
9      zh_en  gemma_owsm4.0-ctc    0.5291                        3.1692   
10     zh_en        seamlessm4t    0.0000                        3.4874   
11     zh_en  tower_owsm4.0-ctc    0.0000                        2.6813   
12     zh_en    aya_owsm4